In [1]:
import cv2
import numpy as np
import glob
import itertools
import pickle
from sklearn.cluster import KMeans
from numpy.linalg import norm
from scipy.io import savemat

### 提取图像的 ORB descriptors

In [2]:
def describeORB(image):
    orb=cv2.ORB_create()
    kp, des=orb.detectAndCompute(image,None)
    return kp,des

In [3]:
path = 'dataset/city_centre/Images'
descriptors=list()

for imagePath in sorted(glob.glob(path+"/*.jpg")):
    print(imagePath)
    im=cv2.imread(imagePath)
    kp,des = describeORB(im)
    descriptors.append(des)

dataset/city_centre/Images/0001.jpg
dataset/city_centre/Images/0002.jpg
dataset/city_centre/Images/0003.jpg
dataset/city_centre/Images/0004.jpg
dataset/city_centre/Images/0005.jpg
dataset/city_centre/Images/0006.jpg
dataset/city_centre/Images/0007.jpg
dataset/city_centre/Images/0008.jpg
dataset/city_centre/Images/0009.jpg
dataset/city_centre/Images/0010.jpg
dataset/city_centre/Images/0011.jpg
dataset/city_centre/Images/0012.jpg
dataset/city_centre/Images/0013.jpg
dataset/city_centre/Images/0014.jpg
dataset/city_centre/Images/0015.jpg
dataset/city_centre/Images/0016.jpg
dataset/city_centre/Images/0017.jpg
dataset/city_centre/Images/0018.jpg
dataset/city_centre/Images/0019.jpg
dataset/city_centre/Images/0020.jpg
dataset/city_centre/Images/0021.jpg
dataset/city_centre/Images/0022.jpg
dataset/city_centre/Images/0023.jpg
dataset/city_centre/Images/0024.jpg
dataset/city_centre/Images/0025.jpg
dataset/city_centre/Images/0026.jpg
dataset/city_centre/Images/0027.jpg
dataset/city_centre/Images/0

In [4]:
#flatten list       
descriptors = list(itertools.chain.from_iterable(descriptors))
#list to array
descriptors = np.asarray(descriptors)

In [6]:
#writting the output
desc_file="descriptors_citycentre/descriptorORB.pickle"

with open(desc_file, 'wb') as f:
    pickle.dump(descriptors, f)

### 由之前提取的 descriptors 聚类得到 dictionary

In [7]:
with open(desc_file, 'rb') as f:
    descriptors=pickle.load(f)

In [8]:
k = 64
visualDictionary = KMeans(n_clusters=k,init='k-means++',tol=0.0001).fit(descriptors)

In [9]:
dict_file="visualDictionary_citycentre/visualDictionary2ORB.pickle"

with open(dict_file, 'wb') as f:
    pickle.dump(visualDictionary, f)

### 计算每幅图片的 vlad desc

In [10]:
def VLAD(X,visualDictionary):
    # 每个 desc 属于哪个 label
    predictedLabels = visualDictionary.predict(X)
    # 64 个 center 位置
    centers = visualDictionary.cluster_centers_
    # 每个 sample point 的 label
    labels=visualDictionary.labels_
    # k == 64
    k=visualDictionary.n_clusters
   
    # 500 * 32
    m,d = X.shape
    V=np.zeros([k,d])
    #computing the differences

    # for all the clusters (visual words)
    for i in range(k):
        # if there is at least one descriptor in that cluster
        if np.sum(predictedLabels==i)>0:
            # add the differences
            V[i]=np.sum(X[predictedLabels==i,:]-centers[i],axis=0)
    

    # 一行一行地，扯平
    V = V.flatten()
    # power normalization, also called square-rooting normalization
    V = np.sign(V)*np.sqrt(np.abs(V))

    # L2 normalization
    V = V/np.sqrt(np.dot(V,V))
    return V

In [11]:
with open(dict_file, 'rb') as f:
    visualDictionary=pickle.load(f) 

In [12]:
path = 'dataset/city_centre/Images'
vlad_desc=list()
idImages =list()

for imagePath in sorted(glob.glob(path+"/*.jpg")):
    print(imagePath)
    im=cv2.imread(imagePath)
    kp,des = describeORB(im)
    # 一幅图片对应一个 v
    v=VLAD(des,visualDictionary)
    vlad_desc.append(v)
    idImages.append(imagePath)

#list to array    
V = np.asarray(vlad_desc)

dataset/city_centre/Images/0001.jpg
dataset/city_centre/Images/0002.jpg
dataset/city_centre/Images/0003.jpg
dataset/city_centre/Images/0004.jpg
dataset/city_centre/Images/0005.jpg
dataset/city_centre/Images/0006.jpg
dataset/city_centre/Images/0007.jpg
dataset/city_centre/Images/0008.jpg
dataset/city_centre/Images/0009.jpg
dataset/city_centre/Images/0010.jpg
dataset/city_centre/Images/0011.jpg
dataset/city_centre/Images/0012.jpg
dataset/city_centre/Images/0013.jpg
dataset/city_centre/Images/0014.jpg
dataset/city_centre/Images/0015.jpg
dataset/city_centre/Images/0016.jpg
dataset/city_centre/Images/0017.jpg
dataset/city_centre/Images/0018.jpg
dataset/city_centre/Images/0019.jpg
dataset/city_centre/Images/0020.jpg
dataset/city_centre/Images/0021.jpg
dataset/city_centre/Images/0022.jpg
dataset/city_centre/Images/0023.jpg
dataset/city_centre/Images/0024.jpg
dataset/city_centre/Images/0025.jpg
dataset/city_centre/Images/0026.jpg
dataset/city_centre/Images/0027.jpg
dataset/city_centre/Images/0

In [13]:
vlad_file="VLADdescriptors_citycentre/VLAD_ORB_W64.pickle"

with open(vlad_file, 'wb') as f:
    pickle.dump([idImages, V, path], f)

### 计算两幅图片之间的 vlad 差别

In [14]:
vlad_file="VLADdescriptors_citycentre/VLAD_ORB_W64.pickle"

with open(vlad_file, 'rb') as f:
    VLAD_images = pickle.load(f)

In [15]:
imageIDs = VLAD_images[0]
VLAD_descs = VLAD_images[1]

In [16]:
num_images = len(imageIDs)
num_images

2474

In [17]:
dist_mat = np.ones((num_images//2, num_images//2))

for i in range(0, num_images, 2):
    for j in range(0, num_images, 2):
        dist = VLAD_descs[i] - VLAD_descs[j]
        dist = norm(dist)
        dist_mat[i//2,j//2] = dist

dist_mat /= np.max(np.abs(dist_mat))

In [18]:
dist_mat

array([[0.        , 0.88224047, 0.89964001, ..., 0.93571117, 0.93903265,
        0.92071591],
       [0.88224047, 0.        , 0.90090927, ..., 0.91783072, 0.91059909,
        0.93030678],
       [0.89964001, 0.90090927, 0.        , ..., 0.9311258 , 0.93001193,
        0.93405687],
       ...,
       [0.93571117, 0.91783072, 0.9311258 , ..., 0.        , 0.93115842,
        0.95026528],
       [0.93903265, 0.91059909, 0.93001193, ..., 0.93115842, 0.        ,
        0.92894859],
       [0.92071591, 0.93030678, 0.93405687, ..., 0.95026528, 0.92894859,
        0.        ]])

In [19]:
savemat("confusion_matrix/citycentre_first.mat", {'D': dist_mat})